In [1]:
import os

In [2]:
%pwd

'd:\\End to End Text Summarizer\\Text-Summarizer\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\End to End Text Summarizer\\Text-Summarizer'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    sourrce_URL: str
    local_data_file: Path
    unzip_dir: Path

In [7]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories


In [14]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(str(config_filepath))
        self.params = read_yaml(str(params_filepath))
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            sourrce_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

In [15]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size


In [16]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            file_name,headers = request.urlretrieve(
                url=self.config.sourrce_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"Downloaded file: {file_name} with following headers: {headers}")
        else:
            logger.info(f"File already exists at: {Path(self.config.local_data_file)}")
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [19]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-06-08 23:47:19,729: INFO: common: YAML file config\config.yaml loaded successfully.]
[2025-06-08 23:47:19,732: INFO: common: YAML file params.yaml loaded successfully.]
[2025-06-08 23:47:19,734: INFO: common: Created directory: artifacts]
[2025-06-08 23:47:19,735: INFO: common: Created directory: artifacts/data_ingestion]


[2025-06-08 23:47:21,396: INFO: 3258841061: Downloaded file: artifacts/data_ingestion/data.zip with following headers: Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: D945:2478D3:1C93CA:4639D6:6845D3B0
Accept-Ranges: bytes
Date: Sun, 08 Jun 2025 18:17:21 GMT
Via: 1.1 varnish
X-Served-By: cache-hyd1100028-HYD
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1749406641.610634,VS0,VE660
Vary: Authorization,Accept-Encoding
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: 8a3eee506a6eb8992789d11bcfa7992c0abc7c5c
Expires: Sun, 08 Jun 2025 18:22:21 GMT
Source-Age: 0

]
